# Key-Value Attention Mechanism Homework on Keras: Character-level Machine Translation (Many-to-Many, encoder-decoder)

In this homework, you will create an MT model with key-value attention mechnism that coverts names of constituency MP candidates in the 2019 Thai general election from Thai script to Roman(Latin) script. E.g. นิยม-->niyom 

In [1]:
# !wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass

# import tensorflow as tf
# print(tf.__version__)

# import matplotlib as mpl
# from matplotlib import font_manager
# mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf') # 3.2+
# mpl.rc('font', family='TH Sarabun New')

In [2]:
%matplotlib inline
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import numpy as np

import random

## Load Dataset
We have generated a toy dataset using names of constituency MP candidates in 2019 Thai General Election from elect.in.th's github(https://github.com/codeforthailand/dataset-election-62-candidates) and tltk (https://pypi.org/project/tltk/) library to convert them into Roman script.

<img src="https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW8/images/dataset_diagram.png" alt="Drawing" style="width: 500px;"/>


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# import shutil
# shutil.copy("/content/drive/MyDrive/FRA 501 IntroNLP&DL/Dataset/mp_name_th_en.csv", "/content/mp_name_th_en.csv")

In [5]:
import csv
with open('Dataset/mp_name_th_en.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    name_th = []
    name_en = []
    for row in readCSV:
        name_th.append(row[0])
        name_en.append(row[1])

In [6]:
for th, en in zip(name_th[:10],name_en[:10]):
    print(th,en)

ไกรสีห์ kraisi
พัชรี phatri
ธีระ thira
วุฒิกร wutthikon
ไสว sawai
สัมภาษณ์  samphat
วศิน wasin
ทินวัฒน์ thinwat
ศักดินัย sakdinai
สุรศักดิ์ surasak


## Task1: Preprocess dataset for Keras
* 2 dictionaries for indexing (1 for input and another for output) OK!
* DON'T FORGET TO INCLUDE special token for padding OK!
* DON'T FORGET TO INCLUDE special token for the end of word symbol (output) OK!
* Be mindful of your pad_sequences "padding" hyperparameter. Choose wisely (post-padding vs pre-padding) OK!

In [7]:
#FILL YOUR CODE HERE
print("th : {:d} | en : {:d}".format(len(name_th),len(name_en)))

th : 10887 | en : 10887


In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Reshape, SimpleRNN
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import numpy as np

In [9]:
sequence = [[1], [2, 3], [4, 5, 6]]
b = pad_sequences(sequence, padding='pre')
a = pad_sequences(sequence, padding='post')
print(b)
print(a)


[[0 0 1]
 [0 2 3]
 [4 5 6]]
[[1 0 0]
 [2 3 0]
 [4 5 6]]


In [10]:
print(name_th[0])

ไกรสีห์


In [11]:
input_chars = list(set(''.join(name_th)))
output_chars = list(set(''.join(name_en)))
# +1 for padding +1 for the end of word
data_size, vocab_size = len(name_th), len(input_chars)+2 
output_vocab_size = len(output_chars)+2

print('There are %d lines and %d unique characters in your input data.' % (data_size, vocab_size))
print('There are %d lines and %d unique characters in your input data.' % (data_size, vocab_size))
maxlen_th = len( max(name_th, key=len)) #max input length
print("Max input length:", maxlen_th)
maxlen_en = len( max(name_en, key=len)) #max input length
print("Max output length:", maxlen_en)

There are 10887 lines and 66 unique characters in your input data.
There are 10887 lines and 66 unique characters in your input data.
Max input length: 20
Max output length: 19


In [12]:
sorted_chars= sorted(input_chars)
sorted_output_chars= sorted(output_chars)
sorted_chars.insert(0,"<PAD>") #PADDING for input
sorted_output_chars.insert(0,"<PAD>") #PADDING for output
sorted_chars.insert(1,"</S>") #the end of word for input
sorted_output_chars.insert(1,"</S>") #the end of word for output
#Input
char_to_ix = { ch:i for i,ch in enumerate(sorted_chars) }
ix_to_char = { i:ch for i,ch in enumerate(sorted_chars) } #reverse dictionary
#Output
output_char_to_ix = { ch:i for i,ch in enumerate(sorted_output_chars) }
ix_to_output_char = { i:ch for i,ch in enumerate(sorted_output_chars) } #reverse dictionary

print(ix_to_char)
print(ix_to_output_char)
print("len(input) : %d | len(output) %d" % (len(ix_to_char),len(ix_to_output_char)))

{0: '<PAD>', 1: '</S>', 2: ' ', 3: 'ก', 4: 'ข', 5: 'ค', 6: 'ฆ', 7: 'ง', 8: 'จ', 9: 'ฉ', 10: 'ช', 11: 'ซ', 12: 'ฌ', 13: 'ญ', 14: 'ฎ', 15: 'ฏ', 16: 'ฐ', 17: 'ฑ', 18: 'ฒ', 19: 'ณ', 20: 'ด', 21: 'ต', 22: 'ถ', 23: 'ท', 24: 'ธ', 25: 'น', 26: 'บ', 27: 'ป', 28: 'ผ', 29: 'ฝ', 30: 'พ', 31: 'ฟ', 32: 'ภ', 33: 'ม', 34: 'ย', 35: 'ร', 36: 'ล', 37: 'ว', 38: 'ศ', 39: 'ษ', 40: 'ส', 41: 'ห', 42: 'ฬ', 43: 'อ', 44: 'ฮ', 45: 'ะ', 46: 'ั', 47: 'า', 48: 'ำ', 49: 'ิ', 50: 'ี', 51: 'ึ', 52: 'ื', 53: 'ุ', 54: 'ู', 55: 'เ', 56: 'แ', 57: 'โ', 58: 'ใ', 59: 'ไ', 60: '็', 61: '่', 62: '้', 63: '๊', 64: '๋', 65: '์'}
{0: '<PAD>', 1: '</S>', 2: '-', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'w', 23: 'y'}
len(input) : 66 | len(output) 24


In [13]:
m=10887  # #sample
Tx=maxlen_th # size of input = 20
Ty=maxlen_en # size of output = 19

In [14]:
X = []
for line in name_th:
    temp=[]
    for char in line:
        temp.append(char_to_ix[char])
    temp.append(char_to_ix["</S>"])
    X.append(temp)
Y = []
for line in name_en:
    temp=[]
    for char in line:
        temp.append(output_char_to_ix[char])
    temp.append(char_to_ix["</S>"])
    Y.append(temp)    

print(len(Y),output_vocab_size)
print("Example X :"+str(X))
print("Example Y :"+str(Y))

X = pad_sequences(X,maxlen=maxlen_th,padding = 'pre')
Y = pad_sequences(Y,maxlen=maxlen_en,padding = 'pre')

X= to_categorical(X,vocab_size)
X=X.reshape(data_size,maxlen_th ,vocab_size)

Y= to_categorical(Y,output_vocab_size)
Y=Y.reshape(data_size,maxlen_en ,output_vocab_size)
print("X : %s | Y : %s" % (str(X.shape),str(Y.shape)))

10887 24
Example X :[[59, 3, 35, 40, 50, 41, 65, 1], [30, 46, 10, 35, 50, 1], [24, 50, 35, 45, 1], [37, 53, 18, 49, 3, 35, 1], [59, 40, 37, 1], [40, 46, 33, 32, 47, 39, 19, 65, 2, 1], [37, 38, 49, 25, 1], [23, 49, 25, 37, 46, 18, 25, 65, 1], [38, 46, 3, 20, 49, 25, 46, 34, 1], [40, 53, 35, 38, 46, 3, 20, 49, 65, 1], [55, 43, 3, 43, 35, 53, 19, 1], [5, 33, 38, 46, 3, 20, 49, 65, 1], [3, 49, 21, 21, 49, 24, 46, 10, 1], [3, 35, 49, 61, 33, 1], [43, 48, 30, 35, 1], [35, 46, 16, 1], [37, 50, 35, 45, 10, 46, 34, 1], [40, 53, 37, 35, 35, 19, 1], [40, 53, 32, 47, 30, 1], [24, 25, 49, 25, 23, 65, 24, 35, 1], [40, 47, 37, 49, 21, 35, 50, 1], [24, 46, 13, 37, 35, 49, 25, 23, 35, 65, 1], [36, 46, 20, 20, 47, 1], [59, 30, 35, 46, 21, 25, 65, 1], [10, 54, 55, 3, 50, 34, 35, 21, 49, 1], [24, 25, 32, 46, 23, 35, 1], [35, 30, 50, 30, 7, 38, 65, 1], [33, 47, 35, 49, 40, 47, 1], [24, 25, 37, 46, 18, 25, 65, 1], [35, 46, 10, 15, 45, 1], [25, 49, 35, 46, 16, 1], [40, 33, 30, 7, 39, 65, 1], [40, 53, 35, 49,

In [15]:
print(X[0].shape)
print(Y[0].shape)
print(X[1].shape)
print(Y[1].shape)

(20, 66)
(19, 24)
(20, 66)
(19, 24)


# Attention Mechanism
## Task 2: Code your own (key-value) attention mechnism
* PLEASE READ: you DO NOT have to follow all the details in (Daniluk, et al. 2017). You just need to create a key-value attention mechanism where the "key" part of the mechanism is used for attention score calculation, and the "value" part of the mechanism is used to encode information to create a context vector.  
* Define global variables
* fill code for one_step_attention function
* Hint: use keras.layers.Lambda 
* Hint: you will probably need more hidden dimmensions than what you've seen in the demo


In [16]:
from tensorflow.keras.activations import softmax
from tensorflow.keras.layers import Lambda
from tensorflow import split
def softMaxAxis1(x):
    return softmax(x,axis=1)


In [17]:
#These are global variables (shared layers)
## Fill your code here
## you are allowed to use code in the demo as your template. 

repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)

#Key-values (Hint)
splitter = Lambda(lambda x:split(x, num_or_size_splits=2,axis=2)) #ได้ key กับ value

#Attention function###
fattn_1 = Dense(maxlen_en, activation = "tanh")
fattn_2 = Dense(1, activation = "relu")

#Attention function###
###
activator = Activation(softMaxAxis1, name='attention_scores') 
dotor = Dot(axes = 1)


In [18]:
def one_step_attention(hidden, s_prev): # (hidden,vecter)
    # hit one_step_attention Demoเปลี่ยนตัว A

    #Fill code here
    key, value = splitter(hidden)


    #concat = ...key...
    #context = ...value...

    #return None # return whatever you need to complete this homework 
    s_prev = repeator(s_prev)
    concat = concatenator([key,s_prev])
    # attention function
    e = fattn_1(concat)
    energies =fattn_2(e)
    # calculate attention_scores (softmax)
    attention_scores = activator(energies)
    #calculate a context vector
    context = dotor([attention_scores,value])

    return context,attention_scores,energies

## Task3: Create and train your encoder/decoder model here
* HINT: you will probably need more hidden dimmensions than what you've seen in the demo

In [19]:
n_h = 32 #hidden dimensions for encoder 
n_s = 64 #hidden dimensions for decoder
encoder_LSTM =  Bidirectional(LSTM(n_h, return_sequences=True),input_shape=(-1, Tx, n_h*2))
decoder_LSTM_cell = LSTM(n_s, return_state = True) #decoder_LSTM_cell
output_layer = Dense(output_vocab_size, activation="softmax") #softmax output layer

In [20]:
#FILL CODE HERE :Hint --> heatmap in CNN + GradCAM

def model(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_h -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    vocab_size -- size of the input vocab
    output_vocab_size -- size of the output vocab

    Returns:
    model -- Keras model instance
    """
    
    # Define the input of your model
    X = Input(shape=(Tx, vocab_size))
    # Define hidden state and cell state for decoder_LSTM_Cell
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    # print(type(s))
    # print(type(c))
    # Initialize empty list of outputs
    outputs = list()

    #Encoder Bi-LSTM
    # h = BidirectionaM(n_h, return_sequences=True),input_shape=(-1, Tx, n_h*2))(X)
    h = encoder_LSTM(X)
    #Iterate for Ty steps (Decoding)
    for t in range(Ty):
    
        #Perform one step of the attention mechanism to calculate the context vector at timestep t
        context,attention_scores,energies = one_step_attention(h, s)
       
        # Feed the context vector to the decoder LSTM cell
        s, _, c = decoder_LSTM_cell(context,initial_state=[s,c])
           
        # Pass the decoder hidden output to the output layer (softmax)
        out = output_layer(s)
        
        # Append an output list with the current output
        outputs.append(out)
    
    #Create model instance
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    
    return model

def inference_encoder(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size): # Hint slide 5
   X = Input(shape=(Tx, vocab_size))
   h = encoder_LSTM(X)

   model = Model(inputs=[X],outputs=h)
   return model

def inference_decoder(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size):
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    X = Input(shape=(Tx, vocab_size))
    h = encoder_LSTM(X)
    context, attention_scores, energies = one_step_attention(h, s0)
    # ...decoder_LSTM_cell...
    s, _, c = decoder_LSTM_cell(context,initial_state=[s0,c0])
    out = output_layer(s)

    model = Model(inputs=[h,s0,c0],outputs=[out,s,c,attention_scores,energies])

    return model




In [21]:
#FIT YOUR MODEL HERE
model = model(Tx, Ty, n_h, n_s, vocab_size, output_vocab_size)

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 66)]     0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 20, 64)       25344       ['input_1[0][0]']                
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                                  
 lambda (Lambda)                [(None, 20, 32),     0           ['bidirectional[0][0]',          
                                 (None, 20, 32)]                  'bidirectional[0][0]',      

In [23]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Y.swapaxes(0,1))
# print(len(outputs[0]))
# print(outputs[0])
# print(s0.shape)
# print(c0.shape)

In [24]:
opt=Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy')
model.fit([X, s0, c0], outputs, epochs=20, batch_size=120)


Epoch 1/20
91/91 [==============================] - 41s 48ms/step - loss: 27.9501 - dense_2_loss: 0.2767 - dense_2_1_loss: 0.1940 - dense_2_2_loss: 0.1829 - dense_2_3_loss: 0.1904 - dense_2_4_loss: 0.2237 - dense_2_5_loss: 0.2823 - dense_2_6_loss: 0.3907 - dense_2_7_loss: 0.5646 - dense_2_8_loss: 0.8749 - dense_2_9_loss: 1.3065 - dense_2_10_loss: 1.8473 - dense_2_11_loss: 2.4499 - dense_2_12_loss: 2.8230 - dense_2_13_loss: 2.9854 - dense_2_14_loss: 2.9417 - dense_2_15_loss: 3.1881 - dense_2_16_loss: 2.8338 - dense_2_17_loss: 2.7933 - dense_2_18_loss: 1.6009
Epoch 2/20
91/91 [==============================] - 4s 48ms/step - loss: 23.4304 - dense_2_loss: 0.0160 - dense_2_1_loss: 0.0108 - dense_2_2_loss: 0.0189 - dense_2_3_loss: 0.0307 - dense_2_4_loss: 0.0639 - dense_2_5_loss: 0.1210 - dense_2_6_loss: 0.2275 - dense_2_7_loss: 0.3985 - dense_2_8_loss: 0.7090 - dense_2_9_loss: 1.1342 - dense_2_10_loss: 1.6774 - dense_2_11_loss: 2.2624 - dense_2_12_loss: 2.6296 - dense_2_13_loss: 2.7648 - d

# Thai-Script to Roman-Script Translation
* Task 4: Test your model on 5 examples of your choice including your name! 
* Task 5: Show your visualization of attention scores on one of your example 

In [95]:
#task 4
#fill your code here
def prep_input(input_list):
    X = []
    for line in input_list:
        temp=[]
        for char in line:
            temp.append(char_to_ix[char])
        X.append(temp)
    X = pad_sequences(X,maxlen=maxlen_th)
    X= to_categorical(X,vocab_size)
    X=X.reshape(len(input_list),maxlen_th ,vocab_size)
    
    return X

EXAMPLES = ['รณกร', 'ธนัท', 'เบส', 'ปุ่น', 'ติน']
s0 = np.zeros((len(EXAMPLES), n_s))
c0 = np.zeros((len(EXAMPLES), n_s))
EXAMPLES = prep_input(EXAMPLES)

prediction = model.predict([EXAMPLES , s0, c0])
print(type(prediction))
print(len(prediction))
print(prediction)
prediction = np.swapaxes(prediction,0,1)
prediction = np.argmax(prediction, axis = -1)
print(prediction)

for j in range(len(prediction)):
    output = "".join([ix_to_output_char[int(i)] for i in prediction[j]])
    print(output)


1/1 [==============================] - 0s 24ms/step
<class 'list'>
19
[array([[9.99906540e-01, 1.21905696e-05, 1.27226497e-06, 8.26574535e-07,
        2.47348021e-06, 1.01017156e-06, 2.17559063e-05, 3.88724999e-08,
        1.66627626e-06, 3.10922496e-06, 2.95978384e-06, 3.14235081e-07,
        1.65769984e-06, 4.00748286e-06, 8.82122731e-06, 3.04615764e-06,
        1.89936713e-07, 5.02490650e-07, 1.55686321e-05, 4.27617124e-06,
        2.77928905e-07, 2.63697757e-07, 4.27224586e-06, 2.97128895e-06],
       [9.99933720e-01, 5.90713853e-06, 1.08188010e-06, 6.03042565e-07,
        1.68725435e-07, 1.55166936e-06, 1.84472174e-05, 1.95929140e-08,
        7.59689954e-07, 1.88005890e-06, 6.99797511e-06, 3.45755041e-07,
        1.02847923e-06, 3.14829617e-06, 4.90617913e-06, 1.46691411e-06,
        8.83324418e-08, 1.29926860e-07, 1.17352956e-05, 4.77265701e-07,
        1.97407743e-07, 1.57391753e-08, 3.70689008e-06, 1.64864480e-06],
       [9.99976516e-01, 1.96826591e-06, 2.88306722e-07, 2.16026

### Plot the attention map
* If you need to install thai font: sudo apt install xfonts-thai
* this is what your visualization might look like:
--> https://drive.google.com/file/d/168J5SPSf4NNKj718wWUEDpUbh8QYZKux/view?usp=share_link

In [26]:
# hit attention_scores in one step attention

In [132]:
# EXAMPLES = ['รณกร', 'ธนัท', 'เบส', 'ปุ่น', 'ติน']
# EXAMPLES = ['รณกร','เมฆ']
EXAMPLES = ['รณกร']

inferEncoder_model = inference_encoder(Tx, Ty, n_h, n_s, vocab_size, output_vocab_size)
inferDecoder_model = inference_decoder(Tx, Ty, n_h, n_s, vocab_size, output_vocab_size)
s0 = np.zeros((len(EXAMPLES), n_s))
c0 = np.zeros((len(EXAMPLES), n_s))
s = s0
c = c0
EXAMPLES = prep_input(EXAMPLES)
h = inferEncoder_model.predict(EXAMPLES,verbose=0)


Ty = 19
word = []
for t in range(Ty):
  out,s,c,attention_scores,energies = inferDecoder_model.predict([h,s,c],verbose=0)
  # array=np.array(out)
  prediction = np.swapaxes(out,0,1) #swapaxes นอก loop
  prediction = [i[0] for i in prediction]
  prediction = np.argmax(prediction, axis = -1)
  word.append(prediction)
  # print(prediction)
print(word)
word = np.array(word)
print(word)


output = "".join([ix_to_output_char[int(i)] for i in word])
print(output)



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 16, 15, 15, 16, 15, 1]
[ 0  0  0  0  0  0  0  0  0  0  0  0 19 16 15 15 16 15  1]
<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>sonnon</S>


In [ ]:
#task 5
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family']='TH Sarabun New'  #you can change to other font that works for you
#fill your code here

In [136]:

sns.heatmap(a,xticklabels=th_n, yticklabels=output_list, vmin=0, vmax=1)

NameError: name 'th_n' is not defined